In [3]:
def binomial_like(n, m):
    prod = QQ(1)
    z = QQ(4)*(n**2)
    for k in range(1,m + 1):
        prod *= (z - (2*k - 1)**2)
    return prod / (QQ(2)**(2*m) * (ZZ(m).factorial()))

show(binomial_like(1, 12))

-232376754295310625/17179869184


In [6]:
from scipy.special import j1
from scipy.optimize import brentq
from sage.all import *
import struct

var('n m x Z')

DD = RealField(200)

def triple_double_split(x):
    f64 = RealField(53)
    hi = f64(x)       
    rem1 = x - RealField(300)(hi)
    med = f64(rem1)   
    rem2 = rem1 - RealField(300)(med)
    lo = f64(rem2)       
    return (lo, med, hi)

def dyadic256_from(sign, exponent, mantissa):
    R = RealField(256)
    mant = Integer(mantissa)
    val = QQ(sign * mant) * QQ(1) / (2 ** -exponent)  # = mant * 2^exponent
    return R(val)


def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

def print_dyadic256(value):
    (s, m, e) = RealField(256)(value).sign_mantissa_exponent();
    mask_128 = (1 << 128) - 1
    lo = m & mask_128
    hi = m >> 128
    print("DyadicFloat256 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: u256 {{")
    print(f"              hi: {format_dyadic_hex(hi)},")
    print(f"              lo: {format_dyadic_hex(lo)},")
    print(f"              }}")
    print("},")

# Expansion for P_n(x)
def Pn_asymptotic(n, x, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m) / ((RealField(500)(2)*RealField(500)(x))**(2*m)) for m in range(0, terms) )

# Expansion for Q_n(x)
def Qn_asymptotic(n, x, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m + 1) / ((RealField(500)(2)*RealField(500)(x))**(2*m + 1)) for m in range(terms) )

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    from sage.rings.real_mpfr import RRtoRR
    f = RRtoRR(RealField(190), RealField(53))
    x_hi = f(x)  # convert to f64
    x_lo = f(x - RealField(190)(x_hi))
    return (float(x_lo),float(x_hi))

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = triple_double_split(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

R = LaurentSeriesRing(RealField(300), 'x',default_prec=300)
x = R.gen()

def Pn_asymptotic(n, y, terms=10):
    # now y = 1/x
    return sum( (-1)**m * binomial_like(n, 2*m) / (QQ(2)**(2*m)) * y**(QQ(2)*m) for m in range(terms) )

def Qn_asymptotic(n, y, terms=10):
    return sum( (-1)**m * binomial_like(n, 2*m + 1) / (QQ(2)**(2*m + 1)) * y**(QQ(2)*m + 1) for m in range(terms) )

P = Pn_asymptotic(1, x, 50)
Q = Qn_asymptotic(1, x, 50)

R_series = (-Q/P)

arctan_series_Z = sum([QQ(-1)**k * x**(QQ(2)*k+1) / RealField(700)(RealField(700)(2)*k+1) for k in range(25)])

alpha0_computed = arctan_series_Z(R_series)
print(alpha0_computed)
print("alpha series coefficients")
for i in range(alpha0_computed.prec()):
    if i > 44:
        break
    coeff = alpha0_computed[i]
    if coeff != 0:
#         print_dyadic256(RealField(300)(f"{coeff}"))
        # if i < 16:
        # print(f"\"{RealField(300)(f"{coeff}")}\",")
        print_dyadic(RealField(300)(f"{coeff}"))
        # else:
        # print(f"{double_to_hex(coeff)},")

exps = alpha0_computed.exponents()
coeffs = alpha0_computed.coefficients()

def sqrt_series(s):
    val = S.valuation()
    lc = S[val]  # Leading coefficient
    b = lc.sqrt() * x**(val // 2)
    
    for _ in range(5):
        b = (b + S / b) / 2
        b = b
    return b

S = (P**2 + Q**2).truncate(50)

b = sqrt_series(S).truncate(30)

# print(b)

print("beta series")
for i in range(24):
    coeff = b[i]
    if coeff != 0:
        # print(double_to_hex(RealField(300)(f"{coeff}")) + ",")
        # print_dyadic(RealField(300)(f"{coeff}"))
        print_double_double("", RealField(300)(f"{coeff}"))


-0.375000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 + 0.164062500000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^3 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 - 0.370898437500000000000000000000000000000000000000000000000000000000000000000000000000000000*x^5 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 + 2.36939784458705357142857142857142857142857142857142857142857142857142857142857142857142857*x^7 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^8 - 30.6240119934082031250000000000000000000000000000000000000000000000000000000000000000000000*x^9 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^10 + 659.18522182

In [6]:
from math import pi, factorial

x = DD(pi) / 128
target_error = 2**-240

for n in range(4, 20):
    term = x**(2*n+1) / factorial(2*n+1)
    print(f"n={n}, term={term:.2e}, bits ≈ {-abs(term).log2() if term != 0 else 999}")
    if term < target_error:
        break


n=4, term=3.13e-7, bits ≈ 21.605667854578722504886316284327550652688482798793625630461
n=5, term=1.76e-9, bits ≈ 29.084035309158744625346903596501636816843960639507303201141
n=6, term=6.95e-12, bits ≈ 37.066445269076355483588360030665051088819626370998150518448
n=7, term=2.04e-14, bits ≈ 45.477698527797840636745292557292991611672713719688681861948
n=8, term=4.63e-17, bits ≈ 54.262169110103542561390263404062298995271395924040831437387
n=9, term=8.35e-20, bits ≈ 63.377029366044802934482165550823669465071947231094656460192
n=10, term=1.23e-22, bits ≈ 72.788282624766288087639098077451609987925034579785187803693
n=11, term=1.49e-25, bits ≈ 82.468283940515960732523514204298974820714506152916839821812
n=12, term=1.54e-28, bits ≈ 92.394110372067204126108796843184474710791398268665810993690
n=13, term=1.35e-31, bits ≈ 102.54644533342712734725773116524352200028669281554162043191
n=14, term=1.03e-34, bits ≈ 112.90878899166766609181037864306224060099199486359102715095
n=15, term=6.80e-38, bits ≈ 1

In [10]:
roll = false
for i in range(0, 28, 2):
    if roll:
        print_double_double("", -RealField(300)(1)/RealField(300)(factorial(i)))
    else:
        print_double_double("", RealField(300)(1)/RealField(300)(factorial(i)))
    roll = not roll

(0x0000000000000000, 0x3ff0000000000000),
(0x0000000000000000, 0xbfe0000000000000),
(0x3c45555555555555, 0x3fa5555555555555),
(0x3bef49f49f49f49f, 0xbf56c16c16c16c17),
(0x3b3a01a01a01a01a, 0x3efa01a01a01a01a),
(0xbb3cbbc05b4fa99a, 0xbe927e4fb7789f5c),
(0xbac2aec959e14c06, 0x3e21eed8eff8d898),
(0xba305d6f8a2efd1f, 0xbda93974a8c07c9d),
(0x39a1d8656b0ee8cb, 0x3d2ae7f3e733b81f),
(0xb94eec01221a8b0b, 0xbca6827863b97d97),
(0x387ea72b4afe3c2f, 0x3c1e542ba4020225),
(0x383aebcdbd20331c, 0xbb90ce396db7f853),
(0xb789ada5fcc1ab14, 0x3aff2cf01972f578),
(0x37071c37ebd16540, 0xba688e85fc6a4e5a),


In [73]:
   R = RealField(300)
   π = R.pi()

   # Generate array entries
   print("pub(crate) static SIN_K_PI_OVER_128_F256: [DyadicFloat256; 65] = [")
   for k in range(65):
       value = R(k) * π / R(128)
       print_dyadic256(value.sin())

   print("];")

pub(crate) static SIN_K_PI_OVER_128_F256: [DyadicFloat256; 65] = [
DyadicFloat256 {
    sign: DyadicSign::Pos,
    exponent: 0,
    mantissa: u256 {
              hi: 0x0_u128,
              lo: 0x0_u128,
              }
},
DyadicFloat256 {
    sign: DyadicSign::Pos,
    exponent: -261,
    mantissa: u256 {
              hi: 0xc90aafbd_1b33efc9_c539edcb_fda0cf2c_u128,
              lo: 0x4ca5f3c3_54a11581_9e635806_dbd038d5_u128,
              }
},
DyadicFloat256 {
    sign: DyadicSign::Pos,
    exponent: -260,
    mantissa: u256 {
              hi: 0xc8fb2f88_6ec09f37_6a17954b_2b7c5171_u128,
              lo: 0x216769fc_da0d0ebf_36fc97b1_fef1fdeb_u128,
              }
},
DyadicFloat256 {
    sign: DyadicSign::Pos,
    exponent: -259,
    mantissa: u256 {
              hi: 0x96a90496_70cfae65_f7757409_4d3c35c4_u128,
              lo: 0x12930f6d_ba22f08a_dc9c4943_ba9bd204_u128,
              }
},
DyadicFloat256 {
    sign: DyadicSign::Pos,
    exponent: -259,
    mantissa: u256 {
       

In [12]:
p = var('p')
p = (DD(24) + DD(1/4)) * DD.pi()
# Define the expression
def expr():
    return p - (3/(8*p)) + (3/(128*p**3)) - (1179/(5120*p**5)) + (1951209/(1146880*p**7)) - (223791831/(9175040*p**9))
print(expr())


76.17869958464145757175388613060650100521941417


In [28]:
z = DD('76.178699584641464070955407805740833') - DD('76.178699584641464')
print(z)
print(z / DD(4)*DD.pi())
from mpmath import mp, j1, taylor, expm1
mp.j1(mp.mpf('76.178699584641464'))

7.095540780574083299999999999988596799011215001e-17
5.572824697374581798277133995318844970172780013e-17
5.74848927250931e-18


mpf('5.940128391303172e-16')

In [94]:
b1 = RealField(1568)
twopi = b1(1) / b1.pi()
print(twopi)
c0 = RealField(256)(twopi)
c1 = RealField(256)(twopi - b1(c0))
c2 = twopi - b1(c0) - b1(c1)

print_dyadic256(c0)
print_dyadic256(c1)
print_dyadic256(c2)

print_dyadic256((b1(2)/b1.pi()).sqrt())


0.318309886183790671537767526745028724068919291480912897495334688117793595268453070180227605532506171912145685453515916073785823692229157305755934821463399678458479933874818155146155492793850615377434785792434795323386724780483447258023664760228445399511431880923780173805347912240978821873875688171057446199892886800497344695478919221796646193566149812333972925609398897304375763149573133928482077991748278697219967736198399924885751170342357716862235037534321093095073976
DyadicFloat256 {
    sign: DyadicSign::Pos,
    exponent: -257,
    mantissa: u256 {
              hi: 0xa2f9836e_4e441529_fc2757d1_f534ddc0_u128,
              lo: 0xdb629599_3c439041_fe5163ab_debbc562_u128,
              }
},
DyadicFloat256 {
    sign: DyadicSign::Neg,
    exponent: -514,
    mantissa: u256 {
              hi: 0x91b7238b_7b645a3f_f36da22b_ec5cdbc6_u128,
              lo: 0x3c429c69_dacb1822_efb9415a_28976f6_u128,
              }
},
DyadicFloat256 {
    sign: DyadicSign::Neg,
    exponent: -772,
    man